In [1]:
import mysql.connector
import pandas as pd
import numpy as np

C:\Users\Adam\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:

mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Ad.1234@",
    database="Fleure"
)
mycursor = mydb.cursor()
query = "SELECT * from plant" 
mycursor.execute(query)
results = mycursor.fetchall()
columns = [i[0] for i in mycursor.description]
df = pd.DataFrame(results, columns=columns)


mycursor.close()
mydb.close()


In [3]:
df.head()

,id,nom,famille,descriptions,descriptions_details,vue,vente,image
0,1,astlib,Asteraceae,Star-shaped flower,A perennial plant belonging to the Asteraceae ...,30,20,images/astlib.png
1,2,beliflower,Campanulaceae,Bell-shaped flower,"Native to temperate regions, the beliflower is...",82,72,images/beliflower.png
2,3,black-eyed-susan,Asteraceae,Yellow flower with black center,The black-eyed susan is a bright yellow flower...,103,74,images/black-eyed-susan.png
3,4,calendula,Asteraceae,Medicinal flower,"Calendula, also known as pot marigold, is a me...",56,41,images/calendula.png
4,5,california_poppy,Papaveraceae,California orange flower,The California poppy is a perennial plant prod...,20,9,images/california_poppy.png


In [4]:
df.columns

Index(['id', 'nom', 'famille', 'descriptions', 'descriptions_details', 'vue',
       'vente', 'image'],
      dtype='object')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   id                    16 non-null     int64 
 1   nom                   16 non-null     object
 2   famille               16 non-null     object
 3   descriptions          16 non-null     object
 4   descriptions_details  16 non-null     object
 5   vue                   16 non-null     int64 
 6   vente                 16 non-null     int64 
 7   image                 16 non-null     object
dtypes: int64(3), object(5)
memory usage: 1.1+ KB


In [6]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS

nlp = spacy.load("en_core_web_sm")

def clean_and_extract_tags(text):
    doc = nlp(text.lower())
    tags = [token.text for token in doc if token.text.isalnum() and token.text not in STOP_WORDS]
    return ', '.join(tags)

columns_to_extract_tags_from = ['descriptions', 'descriptions_details', 'famille']

for column in columns_to_extract_tags_from:
    df[column] = df[column].apply(clean_and_extract_tags)

In [9]:
df['Tags'] = df[columns_to_extract_tags_from].apply(lambda row: ', '.join(row), axis=1)


In [11]:
df['Tags']

0     star, shaped, flower, perennial, plant, belong...
1     bell, shaped, flower, native, temperate, regio...
2     yellow, flower, black, center, black, eyed, su...
3     medicinal, flower, calendula, known, pot, mari...
4     california, orange, flower, california, poppy,...
5     popular, bouquet, flower, carnation, clove, pi...
6     common, meadow, flower, daisy, small, white, f...
7     bright, yellow, flower, coreopsis, perennial, ...
8     spring, yellow, flower, daffodil, iconic, spri...
9     common, edible, flower, dandelion, widespread,...
10    flower, varied, colors, iris, elegant, flower,...
11    ancient, fragrant, flower, magnolia, ornamenta...
12    symbolic, flower, love, rose, undoubtedly, kno...
13    sun, facing, flower, sunflower, large, yellow,...
14    colorful, spring, flower, tulip, iconic, sprin...
15    aquatic, flower, water, lily, aquatic, plant, ...
Name: Tags, dtype: object

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [33]:

def flower_recommendations(df, flowers_id, top_n=10):
    # Convertir les stop words en liste
    stop_words_list = list(STOP_WORDS)
    tfidf_vectorizer = TfidfVectorizer(stop_words=stop_words_list)
    tfidf_matrix_content = tfidf_vectorizer.fit_transform(df['Tags'])
    cosine_similarities_content = cosine_similarity(tfidf_matrix_content, tfidf_matrix_content)
    flowers_index = df[df['id'] == flowers_id].index[0]
    similar_flowers = list(enumerate(cosine_similarities_content[flowers_index]))
    similar_flowers = sorted(similar_flowers, key=lambda x: x[1], reverse=True)
    top_similar_flowers = similar_flowers[1:top_n + 1]
    recommended_flowers_indices = [x[0] for x in top_similar_flowers]
    recommended_flowers_details = df.iloc[recommended_flowers_indices][['id', 'nom', 'famille', 'descriptions', 'descriptions_details', 'vue', 'vente', 'image']]
    
    return recommended_flowers_details

In [34]:
recommended_internships = flower_recommendations(df , 1)
recommended_internships

,id,nom,famille,descriptions,descriptions_details,vue,vente,image
1,2,beliflower,campanulaceae,"bell, shaped, flower","native, temperate, regions, beliflower, herbac...",82,72,images/beliflower.png
10,11,iris,iridaceae,"flower, varied, colors","iris, elegant, flower, varied, colors, blue, p...",14,2,images/iris.png
7,8,coreopsis,asteraceae,"bright, yellow, flower","coreopsis, perennial, plant, producing, bright...",75,45,images/coreopsis.png
9,10,dandelion,asteraceae,"common, edible, flower","dandelion, widespread, plant, known, yellow, f...",116,74,images/dandelion.png
2,3,black-eyed-susan,asteraceae,"yellow, flower, black, center","black, eyed, susan, bright, yellow, flower, ch...",103,74,images/black-eyed-susan.png
4,5,california_poppy,papaveraceae,"california, orange, flower","california, poppy, perennial, plant, producing...",20,9,images/california_poppy.png
6,7,common_daisy,asteraceae,"common, meadow, flower","daisy, small, white, flower, yellow, center, c...",41,11,images/common_daisy.png
3,4,calendula,asteraceae,"medicinal, flower","calendula, known, pot, marigold, medicinal, pl...",56,41,images/calendula.png
13,14,sunflower,asteraceae,"sun, facing, flower","sunflower, large, yellow, flower, follows, pat...",44,10,images/sunflower.png
12,13,rose,rosaceae,"symbolic, flower, love","rose, undoubtedly, known, symbolic, flower, as...",74,53,images/rose.png


In [42]:
def top_flowers():
    average_ratings = df.groupby(
        ['id', 'nom', 'famille', 'descriptions', 'descriptions_details', 'vue', 'vente', 'image'],
        as_index=False 
    )['vente'].mean().rename(columns={'vente': 'average_vente'})  

    top_rated_items = average_ratings.sort_values(by=['vue', 'average_vente'], ascending=False)

    rating_base_recommendation = top_rated_items.head(10)
    return rating_base_recommendation

top_flowers()

,id,nom,famille,descriptions,descriptions_details,vue,image,average_vente
9,10,dandelion,asteraceae,"common, edible, flower","dandelion, widespread, plant, known, yellow, f...",116,images/dandelion.png,74.0
2,3,black-eyed-susan,asteraceae,"yellow, flower, black, center","black, eyed, susan, bright, yellow, flower, ch...",103,images/black-eyed-susan.png,74.0
14,15,tulip,liliaceae,"colorful, spring, flower","tulip, iconic, spring, flower, available, wide...",83,images/tulip.png,71.0
1,2,beliflower,campanulaceae,"bell, shaped, flower","native, temperate, regions, beliflower, herbac...",82,images/beliflower.png,72.0
15,16,water_lily,nymphaeaceae,"aquatic, flower","water, lily, aquatic, plant, floating, flowers...",77,images/water_lily.png,66.0
7,8,coreopsis,asteraceae,"bright, yellow, flower","coreopsis, perennial, plant, producing, bright...",75,images/coreopsis.png,45.0
12,13,rose,rosaceae,"symbolic, flower, love","rose, undoubtedly, known, symbolic, flower, as...",74,images/rose.png,53.0
8,9,daffodil,amaryllidaceae,"spring, yellow, flower","daffodil, iconic, spring, flower, characterize...",65,images/daffodil.png,43.0
3,4,calendula,asteraceae,"medicinal, flower","calendula, known, pot, marigold, medicinal, pl...",56,images/calendula.png,41.0
11,12,magnolia,magnoliaceae,"ancient, fragrant, flower","magnolia, ornamental, tree, producing, large, ...",55,images/magnolia.png,33.0
